<a href="https://colab.research.google.com/github/eaNODE/wimpbo/blob/master/imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMP**

In [0]:
# matplotlib , numpy(linalg , numeric computation , matrices) , scikit-learn
# svm , cnn(pytorch/tensorflow/keras) and opencv to destroy all victim images using orcus


# # http://setosa.io/ev/image-kernels/
# https://www.hindawi.com/journals/js/2016/8742920/
# https://github.com/ArtyZiff35/3D_Reconstruction_From_Stereo_Images
# http://mccormickml.com/2014/01/10/stereo-vision-tutorial-part-i/
# https://www.pyimagesearch.com/2016/07/25/convolutions-with-opencv-and-python/
# http://scipy-lectures.org/advanced/image_processing/
# https://github.com/AlexJinlei/Stereo_Vision_Camera
# https://github.com/topics/depth-map
# https://kushalvyas.github.io/calib.html
# https://github.com/topics/depthmap
# https://github.com/topics/disparity-map
# http://www.aishack.in/tutorials/sift-scale-invariant-feature-transform-introduction/
# https://scipython.com/book/chapter-8-scipy/additional-examples/interpolation-of-an-image/
# https://youtu.be/-ZrDjwXZGxI
# https://www.youtube.com/watch?v=2Q4L3MtdAbY
# https://www.youtube.com/playlist?list=PLh6SAYydrIpctChfPFBlopqw-TGjwWf_8
# https://www.youtube.com/playlist?list=PLiHa1s-EL3vjr0Z02ihr6Lcu4Q0rnRvjm
# https://www.pyimagesearch.com/2016/01/11/opencv-panorama-stitching/
# https://opencv-python-tutroals.readthedocs.io

from google.colab import drive
drive.mount('/gdrive')

In [0]:
# ------------------
# CAMERA CALIBRATION
# ------------------

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
from google.colab import drive
drive.mount('/gdrive')


# %matplotlib inline


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*8,3), np.float32)
objp[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('/gdrive/My Drive/calib/*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (8,6), corners, ret)
        #write_name = 'corners_found'+str(idx)+'.jpg'
        #cv2.imwrite(write_name, img)
#         cv2.imshow('img', img)
        plt.imshow(img)
        plt.show()
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Test undistortion on an image
img = cv2.imread('/gdrive/My Drive/calib/2.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
# return camera matrix, distortion coefficients, rotation and translation vectors etc.
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
np.savez('/gdrive/My Drive/calib/calib.npz', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)

dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('/gdrive/My Drive/calib/test_undist.jpg',dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "/gdrive/My Drive/calib/wide_dist_pickle.p", "wb" ) )
dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)


In [0]:
#!/usr/bin/env python

# ------------------
# CAMERA CALIBRATION
# ------------------


# NOTE: run this on ur local machine!

"""
Running
cameracalib.py  <folder> <image type> <num rows> <num cols> <cell dimension>
like calib.py calib png
--h for help
"""


from google.colab import drive
drive.mount('/gdrive')
import numpy as np
import cv2, glob, sys, argparse

#---------------------- SET THE PARAMETERS
nRows = 9
nCols = 6
dimension = 25 #- mm

workingFolder   = "/gdrive/My Drive/calib"
imageType       = 'jpg'
#------------------------------------------

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, dimension, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((nRows*nCols,3), np.float32)
objp[:,:2] = np.mgrid[0:nCols,0:nRows].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

if len(sys.argv) < 6:
        print("\n [-] Not enough inputs are provided. Using the default values.\n\n" \
              " type -h for help")
else:
    workingFolder   = sys.argv[1]
    imageType       = sys.argv[2]
    nRows           = int(sys.argv[3])
    nCols           = int(sys.argv[4])
    dimension       = float(sys.argv[5])

if '-h' in sys.argv or '--h' in sys.argv:
    print("\n IMAGE CALIBRATION GIVEN A SET OF IMAGES")
    print(" call: python calib.py <folder> <image type> <num rows (9)> <num cols (6)> <cell dimension (25)>")
    print("\n The script will look for every image in the provided folder and will show the pattern found." \
          " User can skip the image pressing ESC or accepting the image with RETURN. " \
          " At the end the end the following files are created:" \
          "  - cameraDistortion.txt" \
          "  - cameraMatrix.txt \n\n")

    sys.exit()

# Find the images files
filename    = workingFolder + "/*." + imageType
images      = glob.glob(filename)

print("[+] IMAGES >>> ", len(images))
if len(images) < 9:
    print("[-] Not enough images were found: at least 9 shall be provided!!!")
    sys.exit()



else:
    nPatternFound = 0
    imgNotGood = images[1]

    for fname in images:
        if 'calibresult' in fname: continue
        #-- Read the file and convert in greyscale
        img     = cv2.imread(fname)
        gray    = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        print("[+] Reading image ", fname)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (nCols,nRows),None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            print("[+] Pattern found! Press ESC to skip or ENTER to accept")
            #--- Sometimes, Harris cornes fails with crappy pictures, so
            corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (nCols,nRows), corners2,ret)
            cv2.imshow('img',img)
            # cv2.waitKey(0)
            k = cv2.waitKey(0) & 0xFF
            if k == 27: #-- ESC Button
                print("[+] Image Skipped")
                imgNotGood = fname
                continue

            print("[+] Image accepted")
            nPatternFound = nPatternFound + 1
            # print(nPatternFound)
            objpoints.append(objp)
            imgpoints.append(corners2)

            # cv2.waitKey(0)
        else:
            imgNotGood = fname


cv2.destroyAllWindows()

if (nPatternFound >= 1):
    print("[+] Found %d good images" % (nPatternFound))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

    # Undistort an image
    img = cv2.imread(imgNotGood)
    h,  w = img.shape[:2]
    print("[+] Image to undistort: ", imgNotGood)
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

    # undistort
    mapx,mapy = cv2.initUndistortRectifyMap(mtx,dist,None,newcameramtx,(w,h),5)
    dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    print("[+] ROI: ", x, y, w, h)

    cv2.imwrite(workingFolder + "/calibresult.png",dst)
    print("[+] Calibrated picture saved as calibresult.png")
    print("[+] Calibration Matrix: ")
    print(mtx)
    print("[+] Disortion: ", dist)

    #--------- Save result
    filename = workingFolder + "/cameraMatrix.txt"
    np.savetxt(filename, mtx, delimiter=',')
    filename = workingFolder + "/cameraDistortion.txt"
    np.savetxt(filename, dist, delimiter=',')

    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        mean_error += error

    print("[!] total error: ", mean_error/len(objpoints))

else:
    print("[!] In order to calibrate you need at least 9 good pictures... try again")

In [0]:

# ------------------
# PANORAMA STICHTING
# ------------------

import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import randrange
from google.colab import drive
drive.mount('/gdrive')


%matplotlib inline

img_ = cv2.imread('/gdrive/My Drive/pan/r.jpg')
img1 = cv2.cvtColor(img_,cv2.COLOR_BGR2GRAY)
img = cv2.imread('/gdrive/My Drive/pan/l.jpg')
img2 = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

sift = cv2.ORB_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)

# Apply ratio test
good = []
for m in matches:
  if m[0].distance < 0.8*m[1].distance:
    good.append(m)
matches = np.asarray(good)

if len(matches[:,0]) >= 4:
  src = np.float32([ kp1[m.queryIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
  dst = np.float32([ kp2[m.trainIdx].pt for m in matches[:,0] ]).reshape(-1,1,2)
  H, masked = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)
else:
  raise AssertionError('Can’t find enough keypoints.')

dst = cv2.warpPerspective(img_,H,(img.shape[1] + img_.shape[1], img.shape[0]))
# plt.subplot(122),plt.imshow(dst),plt.title('Warped Image')
# plt.show()
# plt.figure()
dst[0:img.shape[0], 0:img.shape[1]] = img
cv2.imwrite('/gdrive/My Drive/pan/output.jpg',dst)
plt.figure(figsize=(15,15))
plt.imshow(dst)
plt.show()